# The Battle of Neighborhoods by Ankur Gosar
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will create a recommender system for alternate restaurants in **Mumbai, Maharashtra, India**. 
The target audience of this project will be any food delivery apps like **Zomato, UberEats, FoodPanda, Swiggy** or restaurant table reservation apps like **EazyDiner, Zomato, etc.**

The aim of this project is to provide a user with a alternate restaurant options based on the target restaurant of the user when the target restaurant is not delivering at this time or there is no reservation available when they are using any of the above apps for food delivery or table reservation.

We will use our data science powers to generate a relevant recommendation to the user based on the target restaurant, distance from there, pricing, type of cuisine and average user rating. The smarter the recommendations are the more pleasant the experience will be for the user of the website or application.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* distance to target restaurant in the neighborhood
* type of cuisine, picing and average user rating

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their location in every neighborhood will be obtained using **Foursquare API**
* type of cusine, pricing and average user rating will be obtained using venues feature of **Foursquare API**